# The mirroring is quite computational expensive therefore the diagonal blocks get precomputed

In [1]:
import cloudpickle
from scipy.sparse import dok_matrix
from scipy.sparse import tril


def load_dict(path,zeilen,spalten):
    if(spalten > zeilen):
        template = "block_{i}_{j}".format(i=spalten,j=zeilen)
    else:
        template = "block_{i}_{j}".format(i=zeilen,j=spalten)

    file_path = path + '\\' + template
    with open(file_path, 'rb+') as file:
        co_occurences = cloudpickle.load(file)
    
    return co_occurences


def load_co_occurence(path,zeilen,spalten):
    co_occurences = load_dict(path,zeilen,spalten)
    coocurrence = dok_matrix((20000,20000),dtype='i')
    
    coocurrence._update(co_occurences) # dok_matrix updates #7673 pull request

    if spalten > zeilen :
        print('transposing')
        #this is now done in the training code, to decrease amount of non-empty files
        #coocurrence = coocurrence.transpose()
    
    if spalten == zeilen:
        print('mirroring')
        #print(coocurrence.toarray())
        coocurrence = coocurrence + tril(coocurrence,k=-1).transpose()
    
    return coocurrence

In [2]:
import re
import os
import h5py


path = '..\coocurrence_blocks'
path_hdf = r'S:\\base_coocurrence_hdf\\'
regex = r'block_([0-9]{1,})_([0-9]{1,})'
for file_name in os.listdir(path):
    match = re.match(regex, file_name)
    i     = match.group(1)
    j     = match.group(2)
    print(i,j)
    if (i == '0' and j == '0'):
        print('hello')
        pass
    else:
        continue
    print('output for progress ' + str(i)+','+str(j))
    co_occurence_dok = load_co_occurence(path,i,j)
    co_occurence_np  = co_occurence_dok.toarray()
    f = h5py.File( path_hdf +'./co_occurence_{i}_{j}_gzip.hdf5'.format(i=i,j=j), "w")#plus experiment name
    co_occurence_hdf = f.create_dataset("co-ocurrence", (20000, 20000),compression='gzip')
    co_occurence_hdf[:,:] =  co_occurence_np
    f.close()
    print(co_occurence_hdf)

0 0
hello
output for progress 0,0
mirroring
<Closed HDF5 dataset>
0 1
0 2
0 3
0 4
0 5
0 6
0 7
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7


In [3]:
import re
import os
import h5py
import numpy as np

path = '..\coocurrence_blocks'
path_hdf = r'S:\\base_coocurrence_hdf\\'
regex = r'block_([0-9]{1,})_([0-9]{1,})'
for file_name in os.listdir(path):
    match = re.match(regex, file_name)
    i     = match.group(1)
    j     = match.group(2)
    print(i,j)
    if (i == '0' and j == '0'):
        print('hello')
        pass
    else:
        continue
    print('output for progress ' + str(i)+','+str(j))
    co_occurence_dok = load_co_occurence(path,i,j)
    co_occurence_np  = co_occurence_dok.toarray()
    f = h5py.File( path_hdf +'./co_occurence_{i}_{j}_lzf.hdf5'.format(i=i,j=j), "w")#plus experiment name
    co_occurence_hdf = f.create_dataset("co-ocurrence", (20000, 20000),compression='lzf')
    co_occurence_hdf[:,:] =  co_occurence_np
    f.close()
    print(co_occurence_hdf)

0 0
hello
output for progress 0,0
mirroring
<Closed HDF5 dataset>
0 1
0 2
0 3
0 4
0 5
0 6
0 7
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7


In [2]:
import re
import os
import h5py
import numpy as np

path = 'S:\\tmp\\coocurrence_blocks'
path_hdf = r'S:\\base_coocurrence_hdf\\'
regex = r'block_([0-9]{1,})_([0-9]{1,})'
for file_name in os.listdir(path):
    match = re.match(regex, file_name)
    i     = match.group(1)
    j     = match.group(2)
    print(i,j)
    if (i == '0' and j == '0'):
        print('hello')
        pass
    else:
        continue
    print('output for progress ' + str(i)+','+str(j))
    co_occurence_dok = load_co_occurence(path,i,j)
    co_occurence_np  = co_occurence_dok.toarray()
    np.save('test_{i}_{j}.np'.format(i=i,j=j),co_occurence_np)

0 0
hello
output for progress 0,0
mirroring


NameError: name 'co_occurence_hdf' is not defined

In [5]:
import time
import tensorflow as tf
startTime = time.time()

template = "co_occurence_{i}_{j}.hdf5".format(i=1,j=1)
tmp_hf = h5py.File(path_hdf + template, "r+")
coocurrence = tmp_hf.get("co-ocurrence")[:]
tf_co_occurences = tf.convert_to_tensor(coocurrence[:],dtype=tf.dtypes.float32)
tmp_hf.close()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


Execution time in seconds: 0.6707384586334229


In [15]:
import time
import tensorflow as tf
startTime = time.time()

template = "co_occurence_{i}_{j}_gzip.hdf5".format(i=0,j=0)
tmp_hf = h5py.File(path_hdf + template, "r+")
tf_co_occurences = tf.convert_to_tensor(tmp_hf.get("co-ocurrence")[:],dtype=tf.dtypes.float32)
tmp_hf.close()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 6.035322189331055


In [17]:
import time
import tensorflow as tf
startTime = time.time()

template = "co_occurence_{i}_{j}_lzf.hdf5".format(i=0,j=0)
tmp_hf = h5py.File(path_hdf + template, "r+")
tf_co_occurences = tf.convert_to_tensor(tmp_hf.get("co-ocurrence")[:],dtype=tf.dtypes.float32)
tmp_hf.close()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 4.649369955062866


In [13]:
import time
import tensorflow as tf
startTime = time.time()

array = np.load(path_hdf +'test_{i}_{j}.np.npy'.format(i=0,j=0))
tf_co_occurences = tf.convert_to_tensor(array[:])

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

Execution time in seconds: 3.9026272296905518


In [14]:
with open(path_hdf + 'test_0_0.tensor', 'wb+') as file:
            cloudpickle.dump(tf_co_occurences, file)

In [15]:
import time
import tensorflow as tf
startTime = time.time()

with open(path_hdf + 'test_0_0.tensor', 'rb') as file:
            tensor = cloudpickle.load(file)


executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))
print(type(tensor))

Execution time in seconds: 0.5051162242889404
<class 'tensorflow.python.framework.ops.EagerTensor'>
